In [119]:
import argparse
import json
import os
import re
import time
import uuid
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
from lxml import etree

In [2]:
_BASE_URL = 'http://papers.nips.cc'
_CRAWLING_WAIT_TIME = 0.3
_NEURIPS_NAMESPACE = uuid.UUID('5ee6531f-0d79-4cf1-8da6-dc83cb553336')
_FIRST_YEAR = 1988
_PDF_FOLDER = 'pdfs'
_OUTPUT_PAPERS_FILE = 'papers_data.jsons'

In [86]:
class NipsPaper:
    def __init__(self,year,title,doi,url,authors,abstract):
        self.year = year
        self.title = title
        self.doi = doi
        self.abstract = abstract
        self.authors = authors
        self.url = url
    
    
    def to_json(self):
        return {
            'id': self.doi,
            'title': self.title,
            'year' : self.year,
            'url': self.url,
            'abstract': self.abstract,
            'authors': self.authors
        }

In [87]:
def get_conference_links(year_from, year_to):
    base = _BASE_URL + '/paper_files/paper/'
    number_year_from = year_from - _FIRST_YEAR + 1
    number_year_to = year_to - _FIRST_YEAR + 1
    for idx, i in enumerate(range(number_year_from, number_year_to + 1)):
        year = str(year_from + idx)
        url = base + year
        yield (url,year)

        
def get_papers_year(url):
    base = _BASE_URL
    url_request = requests.get(url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    papers_in_html = soup.find_all('a',attrs = {'title':'paper title'})
    for paper in papers_in_html:
        paper_link = base + paper["href"]
        yield paper_link

        
def get_paper_info(paper_url):
    url_request = requests.get(paper_url)
    soup = BeautifulSoup(url_request.content,'html5lib')
    #Paper title
    paper_title = soup.find('title').text
    #Paper authors
    authors_tags = soup.find_all('meta',attrs = {'name':'citation_author'})
    authors = []
    for author in authors_tags:
        authors.append(author['content'])
    #Paper publication date
    date = soup.find('meta',attrs = {'name':'citation_publication_date'})['content']
    #Paper url
    article_url = soup.find('meta',attrs = {'name':'citation_pdf_url'})['content']
    #Abstract 
    abstract_tag = soup.find('h4',text='Abstract')
    abstract_text = ''
    for p in abstract_tag.find_all_next('p'):
        if len(p.text) != 0:
            abstract_text += p.text
            break
    #Building paper ID
    splitting_url = article_url.split('/')
    index_hash = splitting_url.index("file")
    paper_id = splitting_url[index_hash + 1].split('-')[0]
    return NipsPaper(date,paper_title,paper_id,article_url,authors,abstract_text)

In [88]:
dumping_json = []
start_year = 2012
end_year = 2014
cpt = 0
base_folder = 'NIPS_papers_'
for url in tqdm(get_conference_links(start_year,end_year),'getting_conference_links'):
    time.sleep(_CRAWLING_WAIT_TIME)
    output_folder = base_folder + str(start_year + cpt)
    if not os.path.isdir(output_folder):
        os.makedirs(output_folder)
    for paper_link in tqdm(get_papers_year(url[0])):
        time.sleep(_CRAWLING_WAIT_TIME)
        nips_paper = get_paper_info(paper_link)
        dumping_json.append(nips_paper.to_json())
    cpt += 1
with open('dataset.json', 'w') as file:
    json.dump(dumping_json, file, indent=2)

getting_conference_links: 0it [00:00, ?it/s]
0it [00:00, ?it/s]
1it [00:03,  3.28s/it]
2it [00:04,  2.22s/it]
3it [00:06,  1.89s/it]
4it [00:07,  1.73s/it]
5it [00:09,  1.64s/it]
6it [00:10,  1.55s/it]
7it [00:12,  1.51s/it]
8it [00:13,  1.48s/it]
9it [00:14,  1.50s/it]
10it [00:16,  1.52s/it]
11it [00:18,  1.52s/it]
12it [00:19,  1.60s/it]
13it [00:21,  1.63s/it]
14it [00:23,  1.67s/it]
15it [00:24,  1.62s/it]
16it [00:26,  1.59s/it]
17it [00:27,  1.56s/it]
18it [00:29,  1.55s/it]
19it [00:30,  1.51s/it]
20it [00:32,  1.52s/it]
21it [00:33,  1.52s/it]
22it [00:35,  1.49s/it]
23it [00:36,  1.49s/it]
24it [00:38,  1.52s/it]
25it [00:39,  1.51s/it]
26it [00:41,  1.50s/it]
27it [00:42,  1.53s/it]
28it [00:44,  1.55s/it]
29it [00:46,  1.55s/it]
30it [00:47,  1.52s/it]
31it [00:48,  1.49s/it]
32it [00:50,  1.47s/it]
33it [00:51,  1.46s/it]
34it [00:53,  1.45s/it]
35it [00:54,  1.45s/it]
36it [00:56,  1.45s/it]
37it [00:57,  1.43s/it]
38it [00:58,  1.44s/it]
39it [01:00,  1.44s/it]
40it [01:

295it [07:12,  1.39s/it]
296it [07:14,  1.42s/it]
297it [07:15,  1.41s/it]
298it [07:17,  1.40s/it]
299it [07:18,  1.39s/it]
300it [07:19,  1.38s/it]
301it [07:21,  1.39s/it]
302it [07:22,  1.39s/it]
303it [07:24,  1.40s/it]
304it [07:25,  1.41s/it]
305it [07:26,  1.41s/it]
306it [07:28,  1.41s/it]
307it [07:29,  1.43s/it]
308it [07:31,  1.42s/it]
309it [07:32,  1.41s/it]
310it [07:34,  1.41s/it]
311it [07:35,  1.40s/it]
312it [07:36,  1.39s/it]
313it [07:38,  1.38s/it]
314it [07:39,  1.37s/it]
315it [07:40,  1.37s/it]
316it [07:42,  1.41s/it]
317it [07:43,  1.42s/it]
318it [07:45,  1.44s/it]
319it [07:46,  1.47s/it]
320it [07:48,  1.47s/it]
321it [07:49,  1.43s/it]
322it [07:50,  1.42s/it]
323it [07:52,  1.40s/it]
324it [07:53,  1.40s/it]
325it [07:55,  1.41s/it]
326it [07:56,  1.42s/it]
327it [07:58,  1.41s/it]
328it [07:59,  1.42s/it]
329it [08:00,  1.41s/it]
330it [08:02,  1.42s/it]
331it [08:03,  1.43s/it]
332it [08:05,  1.43s/it]
333it [08:06,  1.46s/it]
334it [08:08,  1.44s/it]


218it [05:15,  1.45s/it]
219it [05:16,  1.45s/it]
220it [05:18,  1.45s/it]
221it [05:20,  1.74s/it]
222it [05:22,  1.72s/it]
223it [05:23,  1.68s/it]
224it [05:25,  1.59s/it]
225it [05:26,  1.52s/it]
226it [05:27,  1.47s/it]
227it [05:29,  1.46s/it]
228it [05:30,  1.46s/it]
229it [05:32,  1.44s/it]
230it [05:33,  1.42s/it]
231it [05:34,  1.40s/it]
232it [05:36,  1.42s/it]
233it [05:37,  1.44s/it]
234it [05:39,  1.44s/it]
235it [05:40,  1.42s/it]
236it [05:41,  1.40s/it]
237it [05:43,  1.41s/it]
238it [05:44,  1.41s/it]
239it [05:46,  1.45s/it]
240it [05:47,  1.45s/it]
241it [05:49,  1.48s/it]
242it [05:50,  1.49s/it]
243it [05:52,  1.46s/it]
244it [05:53,  1.45s/it]
245it [05:55,  1.43s/it]
246it [05:56,  1.45s/it]
247it [05:57,  1.44s/it]
248it [05:59,  1.46s/it]
249it [06:00,  1.43s/it]
250it [06:02,  1.45s/it]
251it [06:07,  2.42s/it]
252it [06:08,  2.22s/it]
253it [06:10,  2.14s/it]
254it [06:12,  1.90s/it]
255it [06:13,  1.76s/it]
256it [06:14,  1.67s/it]
257it [06:16,  1.60s/it]


151it [03:59,  1.56s/it]
152it [04:01,  1.51s/it]
153it [04:03,  1.83s/it]
154it [04:05,  1.75s/it]
155it [04:06,  1.66s/it]
156it [04:08,  1.64s/it]
157it [04:09,  1.56s/it]
158it [04:11,  1.52s/it]
159it [04:12,  1.52s/it]
160it [04:14,  1.54s/it]
161it [04:16,  1.77s/it]
162it [04:20,  2.32s/it]
163it [04:21,  2.09s/it]
164it [04:23,  1.94s/it]
165it [04:24,  1.81s/it]
166it [04:26,  1.71s/it]
167it [04:27,  1.64s/it]
168it [04:29,  1.62s/it]
169it [04:30,  1.59s/it]
170it [04:32,  1.56s/it]
171it [04:33,  1.55s/it]
172it [04:35,  1.51s/it]
173it [04:36,  1.54s/it]
174it [04:38,  1.54s/it]
175it [04:39,  1.51s/it]
176it [04:41,  1.47s/it]
177it [04:42,  1.45s/it]
178it [04:43,  1.44s/it]
179it [04:45,  1.42s/it]
180it [04:46,  1.43s/it]
181it [04:48,  1.43s/it]
182it [04:49,  1.43s/it]
183it [04:51,  1.42s/it]
184it [04:52,  1.42s/it]
185it [04:53,  1.44s/it]
186it [04:55,  1.43s/it]
187it [04:56,  1.42s/it]
188it [04:58,  1.45s/it]
189it [04:59,  1.42s/it]
190it [05:01,  1.43s/it]


In [98]:
def download_papers(paper_file):
    with open(paper_file, 'r') as file:
        papers_data = json.load(file)
    base_folder = 'papers/'
    for paper in papers_data:
        time.sleep(_CRAWLING_WAIT_TIME)
        year = paper['year']
        pdf_url = paper['url']
        year_folder = os.path.join(base_folder, year)
        os.makedirs(year_folder, exist_ok=True)
        response = requests.get(pdf_url)
        if response.status_code == 200:
            filename = paper['title'].replace(' ', '_')+'.pdf'
            file_path = os.path.join(year_folder, filename)
            with open(file_path, 'wb') as file:
                file.write(response.content)
        

In [97]:
download_papers('dataset.json')

papers/2012/Topology_Constraints_in_Graphical_Models.pdf
papers/2012/Clustering_Aggregation_as_Maximum-Weight_Independent_Set.pdf
papers/2012/FastEx:_Hash_Clustering_with_Exponential_Families.pdf


KeyboardInterrupt: 

In [101]:
!docker run --rm --init --ulimit core=0 -p 8070:8070 lfoppiano/grobid:0.8.0

WARN  [2024-01-04 21:47:27,307] org.hibernate.validator.internal.properties.javabean.JavaBeanExecutable: HV000254: Missing parameter metadata for ResponseMeteredLevel(String, int), which declares implicit or synthetic parameters. Automatic resolution of generic type information for method parameters may yield incorrect results if multiple parameters have the same erasure. To solve this, compile your code with the '-parameters' flag.
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/affiliation-address/model.wapiti"
Model path: /opt/grobid/grobid-home/models/affiliation-address/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/name/header/model.wapiti"
Model path: /opt/grobid/grobid-home/models/name/header/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/name/citation/model.wapiti"
Model path: /opt/grobid/grobid-home/models/name/citation/model.wapiti
[Wapiti] Loading model: "/opt/grobid/grobid-home/models/header/model.wapiti"
Model path: /opt

In [108]:
import requests
grobid_api_url = "http://localhost:8070/api/processFulltextDocument"
pdf_file_path = "papers/2012/Topology_Constraints_in_Graphical_Models.pdf"


def pdf_2_xml(pdf_file_path):
    # Open and read the PDF file
    with open(pdf_file_path, "rb") as file:
        # Prepare the files parameter for the API request
        files = {'input': file}

        # Make the API request to GROBID
        response = requests.post(grobid_api_url, files=files)

    # Check if the request was successful
    if response.status_code == 200:
        # Extracted bibliographic information in XML format
        extracted_data = response.text
        print(extracted_data)
    else:
        print(f"Error {response.status_code}: {response.text}")
        return extracted_data


In [113]:
def extract_email(xml):
    xml_content = xml.replace('<?xml version="1.0" encoding="UTF-8"?>', '')

    # Analyser le contenu XML
   
    root = etree.fromstring(xml_content)
    namespaces = {'tei': 'http://www.tei-c.org/ns/1.0'}

    # Utiliser XPath pour trouver les adresses e-mail
    emails = root.xpath('//tei:email/text()', namespaces=namespaces)
    return emails
def extract_ref_from_pdf(pdf_file_path):
    grobid_url = 'http://localhost:8070/api/processReferences'
    with open(pdf_file_path, "rb") as file:
        # Prepare the files parameter for the API request
        files = {'input': file}
        response = requests.post(grobid_url, files=files)
    if response.status_code == 200:
        # Grobid renvoie les données au format TEI XML
        grobid_response = response.text
        # Analysez ici le XML pour extraire les informations
        return grobid_response
    else:
        return "Erreur lors de la communication avec Grobid."

In [117]:
refs = extract_ref_from_pdf(pdf_file_path)
def extract_titles_and_authors(xml_content):
    # Analyser le contenu XML
    root = etree.fromstring(xml_content)
    references = []

    # Parcourir chaque référence
    for biblStruct in root.xpath('//tei:biblStruct', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
        # Extraire le titre
        title = biblStruct.xpath('.//tei:title[@level="a" and @type="main"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
        title = title[0] if title else "Unknown"

        # Extraire les auteurs
        authors = []
        for author in biblStruct.xpath('.//tei:author/tei:persName', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'}):
            forenames = author.xpath('.//tei:forename[@type="first"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            middlenames = author.xpath('.//tei:forename[@type="middle"]/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            surname = author.xpath('.//tei:surname/text()', namespaces={'tei': 'http://www.tei-c.org/ns/1.0'})
            fullname = " ".join(forenames + middlenames + surname)
            authors.append(fullname)

        references.append({'title': title, 'authors': ', '.join(authors)})

    return references

In [118]:
extract_titles_and_authors(refs)

[{'title': 'Model selection through sparse maximum likelihood estimation for multivariate gaussian or binary data',
  'authors': 'O Banerjee, L El Ghaoui, A Aspremont'},
 {'title': 'Emergence of scaling in random networks',
  'authors': 'A Barabási, R Albert'},
 {'title': 'Covariance selection', 'authors': 'A Dempster'},
 {'title': 'Sparse inverse covariance estimation with the graphical lasso',
  'authors': 'J Friedman, T Hastie, R Tibshirani'},
 {'title': 'Unknown', 'authors': 'J Friedman, T Hastie, R Tibshirani'},
 {'title': 'Estimating high-dimensional directed acyclic graphs with the PC-Algorithm',
  'authors': 'M Kalisch, P Bühlmann'},
 {'title': 'Unknown', 'authors': 'S Lauritzen'},
 {'title': 'Learning scale free networks by reweighted 1 regularization',
  'authors': 'Q Liu, A Ihler'},
 {'title': 'High-dimensional graphs and variable selection with the Lasso',
  'authors': 'N Meinshausen, P Bühlmann'},
 {'title': 'Unknown', 'authors': 'M Newman'},
 {'title': 'Partial correlatio